# 推断
在这个课程中，您将从产品评论和新闻文章中推断情感和主题。

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 读取本地的 .env 文件

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 这是模型输出的随机程度
    )
    return response.choices[0].message["content"]

## 产品评论文本

In [3]:
lamp_review = """
需要一盏漂亮的灯来装饰我的卧室，这个灯还有额外的储物功能，价格也不算太高。很快就收到了。\
在运输过程中，我们的灯的绳子断了，但公司很高兴地寄来了一个新的。也在几天内到达了。\
很容易组装。我缺少一个零件，所以我联系了他们的支持，他们很快就给我补发了缺失的零件！\
Lumina对我来说是一个非常关心客户和产品的优秀公司！
"""

## 情感（积极/消极）

In [4]:
prompt = f"""
以下产品评论的情感是什么，
它用三个反引号分隔？

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


积极的情感。


In [5]:
prompt = f"""
以下产品评论的情感是什么，
它用三个反引号括起来？

请用一个单词回答，要么是“积极的”，\
要么是“消极的”。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

积极的


## 辨别情绪类型

In [6]:
prompt = f"""
识别以下评论作者表达的情感列表。列表中不要超过五个项目。将您的答案格式化为用逗号分隔的小写单词列表。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

高兴, 满意, 赞赏, 感激, 推荐


## 辨别愤怒

In [7]:
prompt = f"""
以下评论的作者是否表达了愤怒？\
评论使用三个反引号分隔。\
请用是或否回答。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

否


## 从客户评论中提取产品和公司名称

In [8]:
prompt = f"""
从评论文本中识别以下项目：
- 评论者购买的物品
- 制造该物品的公司

评论使用三个反引号分隔。将您的响应格式化为具有“Item”和“Brand”作为键的JSON对象。如果信息不存在，请使用“unknown”作为值。尽可能简短地回答。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "灯",
  "Brand": "Lumina"
}


## 同时完成多项任务

In [9]:
prompt = f"""
从评论文本中识别以下项目：
- 情感（积极或消极）
- 评论者是否表达愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司

评论使用三个反引号分隔。将您的响应格式化为JSON对象，"Sentiment"，"Anger"，"Item"和"Brand"作为键。如果信息不存在，请使用"unknown"作为值。尽可能简短地回答。将Anger值格式化为布尔值。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "积极",
    "Anger": false,
    "Item": "灯",
    "Brand": "Lumina"
}


## 推断主题

In [10]:
story = """
最近政府进行了一项调查，询问公共部门员工对他们所在部门的满意度。结果显示，NASA是最受欢迎的部门，满意度达到了95%。

一位NASA员工约翰·史密斯对这一结果发表了评论，他说：“我并不惊讶NASA排名第一。这是一个非常棒的工作场所，有着优秀的人才和令人难以置信的机会。我很自豪能成为这样一个创新组织的一员。”

NASA的管理团队也对这一结果表示欢迎，主任汤姆·约翰逊表示：“我们很高兴听到我们的员工对NASA的工作感到满意。我们有一支才华横溢、专注工作的团队，他们不懈努力地实现我们的目标，看到他们的辛勤工作得到回报真是太棒了。”

调查还显示，社会保障局的满意度最低，只有45%的员工表示对自己的工作满意。政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

## 推断5个主题

In [11]:
prompt = f"""
确定以下由三个反引号分隔的文本中正在讨论的五个主题。\
每个项目只需一到两个单词。\
将您的响应格式化为由逗号分隔的项目列表。\
文本示例：'''{story}'''
"""
response = get_completion(prompt)
print(response)

1. 调查结果
2. NASA的工作环境
3. 约翰·史密斯的评论
4. NASA管理团队的回应
5. 社会保障局的满意度


In [12]:
response.split(sep=',')

['1. 调查结果\n2. NASA的工作环境\n3. 约翰·史密斯的评论\n4. NASA管理团队的回应\n5. 社会保障局的满意度']

In [13]:
topic_list = [
    "nasa", "地方政府", "工程", \
    "员工满意度", "联邦政府"
]

## 为特定主题创建新闻提醒

In [14]:
prompt = f"""
确定以下主题列表中的每个项目是否是以下用三个反引号分隔的文本中的主题。

将您的答案作为列表，对于每个主题，使用0或1表示。

主题列表：{", ".join(topic_list)}

文本示例：'''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa - 1
地方政府 - 0
工程 - 0
员工满意度 - 1
联邦政府 - 1


In [16]:
topic_dict = {i.split(' - ')[0]: int(i.split(' - ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("警告：新的NASA故事！")

警告：新的NASA故事！
